In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%matplotlib notebook

import os
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, Concatenate, UpSampling2D, MaxPool2D, Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow.keras.backend as K

from IPython.display import SVG

import tensorflow_probability as tfp


from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler, Normalizer
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

fig_width = 12
plt.rcParams["font.size"] = 40
plt.rcParams['axes.labelsize'] = 40
plt.rcParams['axes.labelweight'] = 'bold'
#plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
#plt.rcParams.keys()

from codes.utils.HAR_utils import get_activity_index, plot_confusion_matrix, get_Train_Test, timeseries_standardize
from codes.models.attention import Attention2D

>>> Done:
>>> Done: Succesfully imported Utils module
>>> Done: Succesfully imported Utils module
Added attention files


## Transform Data

In [3]:
def split_every_n_rows(dataframe, chunk_size=10000):
    chunks = []
    num_chunks = len(dataframe) // chunk_size + 1

    for index in range(num_chunks):
        chunks.append(dataframe[index * chunk_size:(index+1) * chunk_size])

    return chunks

def create_x_y_t(data, features, reqlen=10000, dummy_lval=10):
    t_X = []
    t_Y = []
    t_times = []
    normalizer = Normalizer()

    for i in range(len(data)):
        if len(data[i]) < reqlen:
            x = np.array(data[i][features].values)
            xz = np.zeros((reqlen-len(x),len(features)))
            xs = normalizer.fit_transform(x)
            sx = np.concatenate((xs,xz))

            y = np.array(data[i]['l_val'].values)
            z = np.array([dummy_lval]*(reqlen-len(y)))
            sy = np.concatenate((y,z))
    
            t = np.array(data[i]['timestamp'].values)
            st = np.concatenate((t,z))
    
            t_X.append([sx])
            t_Y.append([sy])
            t_times.append(st)

        else:
            x = np.array(data[i][features].values)
            xs = normalizer.fit_transform(x)

            y = np.array(data[i]['l_val'].values)

            t = np.array(data[i]['timestamp'].values)

            t_X.append([xs])
            t_Y.append([y])
            t_times.append(t)
    t_X = np.array(t_X)
    t_Y = np.array(t_Y)
    #print('t_X.shape',t_X.shape)

    return t_X, t_Y, t_times

In [4]:
def get_train_test_bird_dev(in_df,shuffle=False,trn='A',reqlen=10000):
    in_df.dropna(inplace=True)
    birds = list(in_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
    features = in_df.columns[2:-2]
    print('features: ', features)

    if trn == 'A':
        selector = random.choice(birds)
    else:
        selector = trn

    df_d = {}
    for bird in birds:
        bird_df = in_df[in_df['animal_tag'] == bird]
        bird_df_l = split_every_n_rows(bird_df, reqlen)
        df_d.update({bird:bird_df_l})
        print(bird)

    print(selector)
    tr_birds = birds
    tr_birds.remove(selector)

    test_data = df_d[selector]
    testX, testY, testtimes = create_x_y_t(test_data, features, reqlen=reqlen, dummy_lval=10)
    testX = np.array(testX)
    testY = np.array(testY)
    times = np.array(testtimes)

    trX = []
    trY = []
    for bird in tr_birds:
        train_data = df_d[bird]
        t_X , t_Y, t_times = create_x_y_t(train_data, features, reqlen=reqlen, dummy_lval=10)
        #t_X = np.array(t_X)
        #t_Y = np.array(t_Y)
        trX.append(t_X)
        trY.append(t_Y)
    trX = np.vstack(trX)
    trY = np.vstack(trY)

    if shuffle:
        p = list(zip(trX,trY))
        random.shuffle(p)
        trX, trY = zip(*p)
        trX = list(trX)
        trY = list(trY)

    trainX = np.array(trX)
    trainY = np.array(trY)

    print('train X(shape): ', trainX.shape)
    print('train Y(shape): ', trainY.shape)
    print('test X(shape): ', testX.shape)
    print('test Y(shape): ', testY.shape)
    print('times(shape): ',times.shape)

    return trainX, testX, trainY, testY, times, selector

In [5]:
def get_train_test_bird(in_df,shuffle=False,trn='A'):
    
    in_df.dropna(inplace=True)
    birds = list(in_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
    max_len = 0

    for bird in birds:
        c_len = len(in_df[in_df['animal_tag'] == bird])
        
        if c_len > max_len:
            print(bird,' : ',c_len,' > ',max_len,' ? ',c_len > max_len)
            max_len = c_len
        print('bird: ',bird,'lenght: ',c_len)
    reqlen = int((int(max_len/512)+1)*512)
    print('reqlen: ',reqlen)

    features = in_df.columns[2:-2]
    X = []
    Y = []
    times = []
    normalizer = Normalizer()

    for bird in birds:
            x = np.array(in_df[in_df['animal_tag'] == bird][features].values)
            #print('x.shape',x.shape)
            xz = np.zeros((reqlen-len(x),len(features)))
            xs = normalizer.fit_transform(x)
            sx = np.concatenate((x,xz))
    
            y = np.array(in_df[in_df['animal_tag'] == bird]['l_val'].values)
            z = np.array([10]*(reqlen-len(y)))
            sy = np.concatenate((y,z))
    
            t = np.array(in_df[in_df['animal_tag'] == bird]['timestamp'].values)
            st = np.concatenate((t,z))
    
            X.append([sx])
            Y.append([sy])
            times.append(st)

    if trn != 1000:
        i=0
        if shuffle == True:
            shuffle == False
            print("Warning: shuffle cannot be used when selecting a specific bird")
        for bird in birds:
            if bird == trn:
                selector = i
            i=1+i
    else:
        selector = random.randint(0,len(birds)-1)

    if shuffle:
        p = list(zip(X,Y))
        random.shuffle(p)
        X, Y = zip(*p)
        X = list(X)
        Y = list(Y)

    X = np.array(X)
    Y = np.array(Y)

    times = np.array(times)
    print('X(shape): ',X.shape)
    print('Y(shape): ',Y.shape)
    print('times(shape): ',times.shape)

    testX = np.array([X[selector]])
    testY = np.array([Y[selector]])
    testtimes=times[selector]
    trainX = np.delete(X,selector,0)
    trainY = np.delete(Y,selector,0)
    
    return trainX,testX,trainY,testY,testtimes,selector

# Network design

In [6]:
def down_block(x, filters, kernel_size=(1, 3), padding="same", strides=1, name="down_"):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    p = MaxPool2D((1, 2),name=name+"M")(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(1, 3), padding="same", strides=1, name="up_"):
    us = UpSampling2D((1, 2),name=name+"U")(x)
    concat = Concatenate()([us, skip])
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(concat)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    return c

def bottleneck(x, filters, kernel_size=(1, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl2")(c)
    c = Dropout(0.5)(c)
    return c

def build_unet_model(height,channels,class_num,f=32):
    
    inputs = Input((1, height, channels))
    
    p0 = inputs
    c1, p1 = down_block(p0, f, name="down_1_")
    c2, p2 = down_block(p1, f*2, name="down_2_")
    c3, p3 = down_block(p2, f*4, name="down_3_")
    p3 = Dropout(0.5)(p3)
    
    bn = bottleneck(p3, f*8)
    
    u1 = up_block(bn, c3, f*4, name="up_1_")
    u2 = up_block(u1, c2, f*2, name="up_2_")
    u3 = up_block(u2, c1, f, name="up_3_")
    
    outputs = Dense(class_num, activation = 'softmax')(u3)
    model = Model(inputs, outputs)
    
    return model

def ApplyUnet(trainX, trainY, testX, testY,TestTimes, activities, nb_epoch=100, fi=32):
    
    target_names,indices = get_activity_index(activities)
    #print(target_names)
    indices.append(len(target_names))
    target_names.append('EPadding')
    
    trainY_b = np_utils.to_categorical(trainY, len(target_names))
    testY_b = np_utils.to_categorical(testY, len(target_names))
    lb = LabelBinarizer()
    
    print(indices)
    lb.fit(indices)
    
    model = build_unet_model(trainX.shape[2],trainX.shape[3],len(target_names),f=fi)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    model.summary()
    
    model_checkpoint = keras.callbacks.ModelCheckpoint('bioattunet-2022_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    history = model.fit(trainX, trainY_b, epochs=nb_epoch, validation_split=0.2,callbacks=[model_checkpoint], verbose=0)
    
    fig, (tr_plt,cm_plt) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6))
    
    tr_plt.plot(history.history['loss'])
    tr_plt.plot(history.history['val_loss'])
    tr_plt.set_title('Model loss')
    tr_plt.set_ylabel('Loss')
    tr_plt.set_xlabel('Epoch')
    tr_plt.legend(['loss', 'val_loss'], loc='upper left')

      
    predict_v = model.predict(testX)
    print(predict_v)
    predict_vec = predict_v[0][0].
    confidences = np.max(predict_vec, axis=1)
    testYY = testY[0][0]
    
    predict_class = np.argmax(predict_vec, axis=1)
    predict = [lb.classes_[x] for x in predict_class]
    
    print("len(testYY)",len(testYY))
    print("len(predict)",len(predict))
    
    testXX = testX[0][0]
    
    for i in range(len(testYY)):
        #print(testYY[i])
        if testYY[i]==(len(target_names)-1):
            testYYY = testYY[:i]
            predict1 = predict[:i]
            testXXX = testXX[:i]
            ttimes = TestTimes[:i]
            Confidences = confidences[:i]
            print(testYY[i])
            print(i)
            break
        else:
            testYYY = testYY
            predict1 = predict
            testXXX = testXX
            ttimes = TestTimes
            Confidences = confidences

              
    print("len(testYYY)",len(testYYY))
    print("len(predict1)",len(predict1))
    
    result = classification_report(testYYY, predict1, labels=indices, target_names=target_names)
    print(result)
    testXX=testX
    cnf_matrix = confusion_matrix(testYYY, predict1)
    plot_confusion_matrix(cnf_matrix, fig, cm_plt, classes=target_names)
    fig.tight_layout()
    fig.show()
    
    Sloss = history.history['loss']

    return predict1, testYYY, testXXX, Confidences, ttimes, Sloss

# Run leave_one_bird_out Cross Validation

In [7]:
def leave_one_bird_out_cv(data_df,save_folder,simclass_d, nb_epoch=200, fi=10):

    data_df.drop(['year'],axis=1,inplace=True)
    data_df = data_df[data_df['label']!='unlabeled']
    birds = list(data_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])

    for bird in birds:
        #trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird(data_df,trn=bird)
        trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird_dev(data_df,trn=bird)
        trainX , testX = timeseries_standardize(trainX, testX)
        predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=nb_epoch, fi=fi)
        test_data_with_conf_pred = pd.DataFrame(data = {'timestamp':Ttimes, 'l_val':testYYY, 'predicted_label':predict1,'confidence':Confidences})
        save_fn = 'res_'+bird+'.csv'
        save_path = os.path.join(save_folder,save_fn)
        test_data_with_conf_pred.to_csv(save_path,index=False)

## Load Year data

In [ ]:
data_df_2018 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2018\l_2018_acc.csv")
data_df_2019 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2019\l_2019_acc.csv")
data_df_2020 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2020\l_2020_acc.csv")
data_df_2021 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2021\l_2021_acc.csv")
data_df_2022 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\l_2022_acc.csv")

In [ ]:
data_df = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2018\l_2018_acc.csv")

In [ ]:
#data_df = pd.concat([data_df_2018, data_df_2019, data_df_2020, data_df_2021, data_df_2022])

In [ ]:
data_df

## Load labels

In [ ]:
simclass_df = pd.read_csv(r'G:\JaimeMorales\Codes\omizunagidori\database\labels\O_labels_df.csv')
simclass_d = simclass_df['label'].to_dict()

## Get results

In [ ]:
save_folder = r"G:\JaimeMorales\Codes\omizunagidori\database\results\baselines\U-Net"

In [ ]:
leave_one_bird_out_cv(data_df,save_folder,simclass_d, nb_epoch=100, fi=10)

# Run induvidual test

## Load Data

In [8]:
data_df = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\l_2022_acc.csv")
data_df

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year
0,LB01,1661544781000,0.011719,-0.105469,0.273438,unlabeled,0,2022
1,LB01,1661544781032,0.105469,-0.199219,0.574219,unlabeled,0,2022
2,LB01,1661544781064,-0.183594,0.175781,0.066406,unlabeled,0,2022
3,LB01,1661544781096,-0.550781,-0.027344,-0.144531,unlabeled,0,2022
4,LB01,1661544781129,-0.363281,-0.132813,-0.035156,unlabeled,0,2022
...,...,...,...,...,...,...,...,...
22942599,LB17,1663492499838,-0.062500,-0.234375,2.093750,unlabeled,0,2022
22942600,LB17,1663492499870,-0.433594,-0.074219,2.031250,unlabeled,0,2022
22942601,LB17,1663492499903,-0.304688,-0.320313,1.878906,unlabeled,0,2022
22942602,LB17,1663492499935,-0.062500,-0.554688,1.621094,unlabeled,0,2022


In [9]:
simclass_df = pd.read_csv(r'G:\JaimeMorales\Codes\omizunagidori\database\labels\O_labels_df.csv')
simclass_d = simclass_df['label'].to_dict()
#simclass_d[len('simclass_d')]='pad_class'
simclass_d

{0: 'unlabeled',
 1: 'stationary',
 2: 'flight_take_off',
 3: 'flight_cruising',
 4: 'unknown',
 5: 'bathing',
 6: 'surface_seizing',
 7: 'preening',
 8: 'foraging_dive',
 9: 'body_shaking'}

In [10]:
data_df.drop(['year'],axis=1,inplace=True)
data_df = data_df[data_df['label']!='unlabeled']

In [11]:
birds = list(data_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
birds

['LB01',
 'LB02',
 'LB03',
 'LB04',
 'LB05',
 'LB08',
 'LB09',
 'LB11',
 'LB13',
 'LB14',
 'LB15',
 'LB17']

## Divide train Test

In [12]:
#trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird(data_df,trn='LB0x')
trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird_dev(data_df, shuffle=False, trn='A', reqlen=10000)

features:  Index(['acc_x', 'acc_y', 'acc_z'], dtype='object')
LB01
LB02
LB03
LB04
LB05
LB08
LB09
LB11
LB13
LB14
LB15
LB17
LB04
train X(shape):  (63, 1, 10000, 3)
train Y(shape):  (63, 1, 10000)
test X(shape):  (9, 1, 10000, 3)
test Y(shape):  (9, 1, 10000)
times(shape):  (9, 10000)


In [13]:
print(selector)

LB04


In [14]:
test_data = data_df[data_df['animal_tag']==selector]
test_data

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val
6052372,LB04,1661805632806,0.453125,-0.140625,0.742188,flight_cruising,3
6052373,LB04,1661805632838,0.523438,-0.078125,1.261719,flight_cruising,3
6052374,LB04,1661805632870,0.085938,-0.515625,1.632813,flight_cruising,3
6052375,LB04,1661805632903,-0.253906,-0.484375,1.722656,flight_cruising,3
6052376,LB04,1661805632935,-0.328125,-0.437500,1.671875,flight_cruising,3
...,...,...,...,...,...,...,...
7817704,LB04,1661902062000,0.000000,0.000000,0.000000,flight_cruising,3
7817705,LB04,1661902062032,0.000000,0.000000,0.000000,flight_cruising,3
7817706,LB04,1661902062064,0.000000,0.000000,0.000000,flight_cruising,3
7817707,LB04,1661902062096,0.000000,0.000000,0.000000,flight_cruising,3


In [15]:
print('trainX(shape): ',trainX.shape)
print('trainY(shape): ',trainY.shape)

print('testX(shape): ',testX.shape)
print('testY(shape): ',testY.shape)
print('testtimes(len): ',len(testtimes))

trainX(shape):  (63, 1, 10000, 3)
trainY(shape):  (63, 1, 10000)
testX(shape):  (9, 1, 10000, 3)
testY(shape):  (9, 1, 10000)
testtimes(len):  9


In [16]:
trainX , testX = timeseries_standardize(trainX, testX)

## Network testing

In [17]:
predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=100, fi=10)
#predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet_dev(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=100, fi=10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 10000, 3  0           []                               
                                )]                                                                
                                                                                                  
 down_1_1 (Conv2D)              (None, 1, 10000, 10  100         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 down_1_2 (Conv2D)              (None, 1, 10000, 10  310         ['down_1_1[0][0]']               
                                )                          

<IPython.core.display.Javascript object>

1/1 [==============================] - 1s 769ms/step
[[[[1.12227909e-02 1.89765573e-01 8.51504281e-02 ... 2.81181317e-02
    1.04850372e-02 9.78230033e-03]
   [5.37777646e-03 3.32477957e-01 2.81567201e-02 ... 1.15396073e-02
    5.68795344e-03 8.38474371e-03]
   [2.85910768e-03 2.40051240e-01 2.13573724e-02 ... 6.61927136e-03
    1.77364354e-03 2.92867306e-03]
   ...
   [1.55496169e-02 3.63408893e-01 1.65256634e-02 ... 2.52240058e-02
    6.33333623e-03 3.78759913e-02]
   [1.96530297e-02 3.11968446e-01 1.99282784e-02 ... 2.93504968e-02
    9.39125568e-03 5.48538454e-02]
   [3.28556634e-02 2.22292572e-01 2.71068718e-02 ... 4.53700498e-02
    1.69609487e-02 7.94675127e-02]]]


 [[[3.52657028e-02 2.01081946e-01 9.19380561e-02 ... 6.22099340e-02
    3.33624110e-02 4.36408408e-02]
   [3.19753513e-02 2.92914748e-01 5.18078879e-02 ... 4.96295132e-02
    1.82122923e-02 4.93715368e-02]
   [2.07207426e-02 3.55306238e-01 3.40987146e-02 ... 4.25591096e-02
    9.86038335e-03 3.31126638e-02]
   ...
  

In [22]:
len(Ttimes)

9

In [23]:
test_data_with_conf_pred = pd.DataFrame(data = {'time':Ttimes[-1], 'l_val':testYYY, 'predicted_label':predict1,'confidence':Confidences})

In [ ]:
save_folder = r'G:\JaimeMorales\Codes\Biodata\database\results'
save_fn = 'res_'+birds[selector]+'.csv'
save_path = os.path.join(save_folder,save_fn)
save_path

In [ ]:
test_data_with_conf_pred.to_csv(save_path,index=False)

In [ ]:
tttimes = range(len(predict1))

In [ ]:
labels,indices = get_activity_index(simclass_d)
indices.append(11)
labels.append('EPadding')
figs = plt.figure('result',figsize=(10, 4))
axs = plt.subplot(111)
axs.plot(tttimes, predict1, color='r')
axs.plot(tttimes, testYYY, color='g')
axs.set_ylabel('Activity label')
axs.set_yticks(indices)
axs.set_yticklabels(labels)
figs.show()